# dev fast tip detection
Tim Tyree<br>
10.5.2020

In [1]:
#pylab
%matplotlib inline
import numpy as np, pandas as pd, matplotlib.pyplot as plt

import skimage as sk
from skimage import measure, filters

from numba import njit, jit, vectorize
from PIL import Image
import imageio
# import cv2 as cv

#automate the boring stuff
from IPython import utils
import time, os, sys, re
beep = lambda x: os.system("echo -n '\\a';sleep 0.2;" * x)
if not 'nb_dir' in globals():
    nb_dir = os.getcwd()
width = 512
height = 512
channel_no = 3

# #load the libraries
from lib import *

#use cuda via numba
from numba import jit, njit, vectorize, cuda, uint32, f8, uint8
from numba.typed import List
# from lib.contours_to_tips import *

%autocall 1
%load_ext autoreload
%autoreload 2

100000
Automatic calling is: Smart


# return an example block of 2x2 pixels with pbc

In [14]:
#load ^that loaded initial after 20000 steps = 500 ms at time_steps of size 0.025
file = '/Users/timothytyree/Documents/GitHub/care/notebooks/Data/test_txt.npz'
txt  = load_buffer(file)
t   = 500 #ms

zero_txt = txt.copy()*0.
width, height = txt.shape[:2]

#compute as discrete flow map dtexture_dt
dtexture_dt = zero_txt.copy()
get_time_step(txt, dtexture_dt)

#compute the images to find isosurfaces of
img    = txt[...,0]
dimgdt = dtexture_dt[...,0]

In [26]:
h    = 0.025 #ms
tmax = 3500#3025.600-2*h# 886.6-h#6000#600#510#600#505.225# #ms

V_threshold = 0.8
# jump_threshold = 2 
# size_threshold = 0
# pad=2
# decimals=10


array1 = img
array2 = dimgdt

level1 = 0.4
level2 = 0.0

In [18]:
#for each window
r0 = 5
c0 = 10
# if r1>r0 and c1>c0:
win = array1[r0:r0+2,c0:c0+2]
#else handle the edge case

In [23]:
assert (win.shape == (2, 2))

In [24]:
#test based dev of needed functions
# def compute_segments_in_window(win):
#     pass
def compute_segments_in_window(ul,ll,ur,ul):
    pass
    #TODO:     make a cythonic solution  in sublime
    #TODO: break ^this up into smaller functions
    
def compute_theta(u,v):
    pass
    #how is a segment structured?

def compute_intersect(u,v):
    pass
    #how is a segment structured?
def _box(u,v):
    originx = np.min(u[0])
    #how is a segment structured?
def _is_point_in_box(pt,box):
    pt[0] >= box
    #how is a box structured?



SyntaxError: unexpected EOF while parsing (<ipython-input-24-d568b4cadeac>, line 7)

In [25]:
segments = compute_segments_in_window(ul,ll,ur,ul,level)

NameError: name 'compute_segments_in_window' is not defined